<a id="1"></a>
# <div style="text-align:center; border-radius:15px 50px; padding:15px; color:white; margin:0; font-size:150%; font-family:Pacifico; background-color:#2a6199; overflow:hidden"><b> 🧠 🧠  TP2 - Elagage (Pruning) des modèles IA  </b> 

Ce notebook illustre l'application des techniques d'**élagage** sur des modèles d'IA dans le but d'améliorer leurs performances sur les ensembles de données **MNIST** et **Fashion-MNIST**. et de réduire leur taille. Au cours de ce lab, vous découvrirez :


1. Comprendre et appliquer différentes techniques d'élagage
   - Comment distinguer les différentes granularités de l'élagage, telles que :
     - **Élagage fine-grainé/informel** : suppression individuelle des poids.
     - **Élagage structuré** : suppression de structures entières comme les canaux ou les filtres.
   - Les principaux **critères d'élagage** utilisés pour sélectionner les poids ou structures à supprimer, y compris :
     - Les normes L1 et L2 pour mesurer l'importance des poids.
     - L'utilisation du **pourcentage de zéros** pour quantifier la parcimonie dans les poids.
   - L'impact de la **sensibilité à l'élagage** en modifiant les ratios de suppression et en analysant leurs effets sur les performances du modèle.

2. Évaluer l'impact de l'élagage sur les performances et l'efficacité du modèle
   - La manière dont l'élagage influence la **précision** des modèles, en évaluant leurs performances sur des ensembles d'entraînement et de validation.
   - Comment mesurer l'impact sur l'**efficacité computationnelle**, en examinant :
     - La réduction des **FLOPs** (opérations flottantes par seconde).
     - L'utilisation de la mémoire et l'empreinte du modèle.
     - Le **temps d'inférence** avant et après l'élagage.


  ---
### <a id='toc1_1_1_'></a>[**Table des matières**](#toc0_)


- [ 🧠  TP2 - Elagage (Pruning) des modèles IA ](#toc1_)    
    
- [1. Introduction à l'élagage (Pruning)](#toc2_)    
  - [1.1 Techniques d'élagage ](#toc3_)    
        - [**Impact sur les performances**](#toc3_1_1_1_)    
  - [1.2 Granularités d’élagage ](#toc4_)    
  - [1.3 Critères d'élagage](#toc5_)    
    - [a) Élagage basé sur l'amplitude ](#toc6_)    
    - [b) Élagage Basé sur le Pourcentage Moyen de Zéros (PMZ) - Average Percentage of Zeros (APoZ)](#toc7_)    
- [🔍 2. Échauffement : Bases de l'Elagage](#toc8_)    

  
- [🖼️ 3. Exploration de l'élagage des modèles IA](#toc9_)    
  - [3.1 Chargement des données](#toc10_)    
  - [3.2 Construire un modèle](#toc11_)    
  - [3.3 Entraîner le modèle sans élagage](#toc12_)      
  - [3.4. Évaluation de la Précision du modèle de base  📊](#toc13_)    
  - [3.4 Appliquer l'Élagage](#toc14_)    
  - [3.5 Compresser le modèle élagué](#toc15_)    
  - [3.6  Combinez l'élagage (**pruning**) et la quantification (**quantization**) pour créer un modèle 10 fois plus petit.](#toc17_)    

- [✅  4. Conslusion](#toc19_)    

- [🎁 5. BONUS](#toc20_)    

  ---

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->


<div style="background-color:#eaf6fb; border-radius:8px; padding:20px; border-left:6px solid #4682b4; color:black; font-family:Arial, sans-serif;">

# <a id='toc2_'></a>[<div style="text-align:center; border-radius:8px; padding:10px; color:white; margin:10px 0; font-size:110%; font-family:Arial, sans-serif; background-color: #4682b4;"><b>1. Introduction à l'élagage (Pruning)</b></div>](#toc0_)



L'optimisation des réseaux de neurones est essentielle pour améliorer leur efficacité et réduire les coûts de calcul. Une méthode efficace est l'**élagage des modèles**, qui consiste à supprimer des connexions ou des neurones dans le réseau, réduisant ainsi le nombre total de paramètres et accélérant les calculs.

<img src="https://github.com/bouzid-s/IASSC/blob/main/sources/download.png?raw=true" alt="Exemple d'élagage" width="500">

En général, les réseaux de neurones ont une structure dense comme celle de gauche, où chaque neurone est connecté à tous les neurones de la couche suivante. Cette connectivité dense engendre un grand nombre de multiplications en virgule flottante. En réduisant ces connexions et en favorisant la parcimonie, nous pouvons optimiser le réseau pour des calculs plus rapides et une utilisation réduite des ressources.


<div style="background-color:#eaf6fb; border-radius:8px; padding:20px; border-left:6px solid #f17c12; color:black; font-family:Arial, sans-serif;">

# <a id='toc3_'></a>[<div style="text-align:left; border-radius:6px; padding:px; color:white; margin:5px 0; font-size:70%; font-family:Arial, sans-serif; background-color: #f17c12;"><b>1.1 Techniques d'élagage </b></div>](#toc0_)



1. **Élagage des poids** :
   - Supprime des connexions individuelles en mettant à zéro certains poids de la matrice des poids.
   - Pour atteindre un niveau de parcimonie de $k\%$, les poids dans la matrice $W$ sont classés par leur magnitude ($|w_{i,j}|$), et les $k\%$ plus petits sont mis à zéro.

2. **Élagage des neurones** :
   - Supprime des neurones entiers en mettant à zéro des colonnes entières dans la matrice des poids.
   - Pour atteindre un niveau de parcimonie de $k\%$, les colonnes de la matrice des poids sont classées selon leur norme $L_2$ ($|w| = \sqrt{\sum_{i=1}^{N} (x_i)^2}$), et les $k\%$ plus petites sont supprimées.

#### <a id='toc3_1_1_1_'></a>[**Impact sur les performances**](#toc0_)

Au fur et à mesure que le niveau de parcimonie augmente et que davantage de connexions ou de neurones sont supprimés, les performances du réseau sur la tâche donnée se dégradent progressivement.
</div>

<div style="background-color:#eaf6fb; border-radius:8px; padding:20px; border-left:6px solid #f17c12; color:black; font-family:Arial, sans-serif;">

# <a id='toc4_'></a>[<div style="text-align:left; border-radius:6px; padding:px; color:white; margin:5px 0; font-size:70%; font-family:Arial, sans-serif; background-color: #f17c12;"><b>1.2 Granularités d’élagage </b> </div>](#toc0_)

  Ce schéma illustre différentes granularités d'élagage couramment utilisées dans les modèles d'IA, organisées sur un continuum entre les approches **irrégulières** et **régulières** :
  1. **Fine-grained Pruning** (élagage au niveau des poids) :
    - Chaque poids est évalué individuellement et peut être supprimé indépendamment.
    - Offre une grande flexibilité mais introduit une irrégularité structurelle.
  2. **Channel-level Pruning** :
    - Élimine des canaux complets des tenseurs de sortie.
    - Favorise des gains importants en performances et en réduction de taille tout en étant plus régulier.

  <img src="https://github.com/bouzid-s/IASSC/blob/main/sources/fine_gran_all0.png?raw=true" alt="typesQuant" width="700">

  **Légende :**
    - **Rouge** : parties préservées du modèle.
    - **Blanc** : parties supprimées par l'élagage.

  **Applications :**
  - Les approches **irrégulières** conviennent pour maximiser la compression mais nécessitent des optimisations logicielles spécifiques.
  - Les approches **régulières** facilitent une implémentation efficace sur des architectures matérielles, comme les GPUs.
</div>


<div style="background-color:#eaf6fb; border-radius:8px; padding:20px; border-left:6px solid #f17c12; color:black; font-family:Arial, sans-serif;">

# <a id='toc5_'></a>[<div style="text-align:left; border-radius:6px; padding:6px; color:white; margin:5px 0; font-size:70%; font-family:Arial, sans-serif; background-color:#f17c12;"><b>1.3 Critères d'élagage</b></div>](#toc0_)

<div style="background-color:rgba(149, 177, 186, 0.2); border-radius:8px; padding:15px; border-left:6px solid #4682b4; color:black; font-family:Arial, sans-serif;">

# <a id='toc6_'></a>[<div style="text-align:left; border-radius:6px; padding:6px; color:white; margin:5px 0; font-size:50%; font-family:Arial, sans-serif; background-color:#115175;"><b> a) Élagage basé sur l'amplitude </b></div>](#toc0_)


### <a id='toc6_1_1_'></a>[Critère d'élagage basé sur la norme $L_p$](#toc0_)

Un critère efficace pour l'élagage repose sur la norme $L_p$, qui mesure l'amplitude des poids d'un modèle. Cette méthode considère que les poids ayant une contribution significative (normes élevées) sont plus importants que les autres.

#### <a id='toc6_1_1_1_'></a>[**Définition de la norme $L_p$ :**](#toc0_)
La norme $L_p$ est définie par la formule suivante :

$
\|W^{(S)}\|_p = \left( \sum_{i \in S} |w_i|^p \right)^{\frac{1}{p}}
$

où :
- $W^{(S)}$ représente un ensemble de paramètres (éléments ou lignes d'une matrice).
- $p$ détermine le type de norme ($p = 1$ pour $L_1$, $p = 2$ pour $L_2$, etc.).

---

#### <a id='toc6_1_1_2_'></a>[**Cas pratiques :**](#toc0_)

1. **Norme $L_1$ élémentaire** :
   - Chaque poids individuel est évalué par sa valeur absolue ($|w_i|$).
   - Les poids ayant les plus faibles valeurs absolues sont mis à zéro.

2. **Norme $L_1$ par ligne** :
   - La somme des valeurs absolues d'une ligne est utilisée comme critère.
   - Exemple : Pour une ligne $[3, -2]$, la norme $L_1$ est $|3| + |-2| = 5$.

3. **Norme $L_p$ par ligne** :
   - Une ligne est élaguée selon sa norme $L_p$, calculée comme suit :
     \[
     \|W_{\text{ligne}}\|_p = \left( \sum_{i=1}^{n} |w_i|^p \right)^{\frac{1}{p}}
     \]
   - Exemple ($p = 2$, norme $L_2$) : Pour la ligne $[3, -2]$, la norme est $\sqrt{3^2 + (-2)^2} = \sqrt{13}$.

---

#### <a id='toc6_1_1_3_'></a>[**Exemple détaillé : Norme $L_2$ par ligne**](#toc0_)

| **Poids**       | **Norme $L_2$ (par ligne)** | **Poids élagués** |
|------------------|-----------------------------|-------------------|
| $3, -2$        | $\sqrt{3^2 + (-2)^2} = \sqrt{13}$ | $0, 0$         |
| $1, -5$        | $\sqrt{1^2 + (-5)^2} = \sqrt{26}$ | $1, -5$        |

---

#### <a id='toc6_1_1_4_'></a>[**Interprétation :**](#toc0_)
- **$L_1$ élémentaire** : Simplifie la matrice en supprimant les poids isolés les moins importants.
- **$L_1$ ou $L_p$ par ligne** : Évalue l'importance d'une structure (ligne ou canal) pour une suppression plus structurée.
- Ces méthodes permettent de réduire la complexité du modèle tout en conservant les informations essentielles pour les prédictions.
---
</div>
<div style="background-color:rgba(149, 177, 186, 0.2); border-radius:8px; padding:15px; border-left:6px solid #4682b4; color:black; font-family:Arial, sans-serif;">

# <a id='toc7_'></a>[<div style="text-align:left; border-radius:6px; padding:6px; color:white; margin:5px 0; font-size:50%; font-family:Arial, sans-serif; background-color: #115175;"><b>  b) Élagage Basé sur le Pourcentage Moyen de Zéros (PMZ) - Average Percentage of Zeros (APoZ) </b></div>](#toc0_)




 **Définition** :
Le **Pourcentage Moyen de Zéros (PMZ).** est une métrique utilisée pour mesurer l'importance des neurones dans un réseau neuronal. Ce critère est basé sur le pourcentage de sorties égales à zéro pour chaque neurone.

- **Idée principale** : Plus le **APoZ** d'un neurone est faible, plus ce neurone est considéré comme important.
- Cette méthode permet de détecter les neurones sous-utilisés ou non essentiels, qui peuvent être supprimés pour réduire la complexité du modèle.



**Illustration :**
1. Les **activations de sortie** d'un réseau convolutif sont représentées par des matrices pour chaque canal.
2. Pour chaque canal, le **APoZ** est calculé comme suit :
   - $ \text{APoZ} = \frac{\text{Nombre de zéros}}{\text{Nombre total d'éléments}} $.



**Exemple de Calcul :**


Pour un réseau avec une sortie de dimension $ \text{Width} = 4 $, $ \text{Height} = 4 $, $ \text{Channel} = 3 $, $ \text{Batch} = 2 $ :

<img src="https://github.com/bouzid-s/IASSC/blob/main/sources/zero_prun2.png?raw=true" alt="typesQuant" width="700">


- **Canal 0** :
  - Nombre de zéros : $ 5 + 6 = 11 $.
  - Total d'éléments : $ 2 \times 4 \times 4 = 32 $.
  - $ \text{APoZ} = \frac{11}{32} $.

- **Canal 1** :
  - $ \text{APoZ} = \frac{12}{32} $.

- **Canal 2** :
  - $ \text{APoZ} = \frac{14}{32} $ (neurone élagué en raison d'un APoZ élevé).


**Conclusion :**
L'APoZ fournit une méthode efficace pour identifier les neurones inutiles en analysant leurs activations. Les neurones présentant un **APoZ élevé** peuvent être élagués sans perte significative de précision, permettant de réduire la taille et la complexité du modèle.

---
</div>
</div>
</div>
  


<div style="background-color:#eaf6fb; border-radius:8px; padding:20px; border-left:6px solid #4682b4; color:black; font-family:Arial, sans-serif;">

# <a id='toc8_'></a>[<div style="text-align:center; border-radius:8px; padding:8px; color:white; margin:10px 0; font-size:100%; font-family:Arial, sans-serif; background-color:#4682b4;"><b>🔍 2. Échauffement : Bases de l'Elagage</b></div>](#toc0_)




 
<div style="background-color:#fae1e1; border-radius:8px; padding:15px; border-left:6px solid#bf2929; color:black; font-family:Arial, sans-serif;">

### <a id='toc8_1_1_'></a>[**Consigne : Développer des fonctions d'élagage selon les normes $L_1$ (éléments et lignes) et $L_2$ (lignes)**](#toc0_)


Implémentez trois fonctions distinctes pour appliquer un élagage fine-grainé à une matrice 2D en fonction des critères suivants :
1. **Norme $L_1$ élémentaire** : Mettre à Zero les éléments ayant les plus faibles valeurs absolues.
2. **Norme $L_1$ par ligne** : Mettre à Zero les lignes ayant les plus faibles sommes des valeurs absolues.
3. **Norme $L_2$ par ligne** : Mettre à Zero les lignes ayant les plus faibles normes $L_2$ (euclidiennes).

---

##### <a id='toc8_1_1_1_1_'></a>[**Paramètres des fonctions** :](#toc0_)
- `matrix` : Matrice 2D NumPy à élaguer.
- `target_sparsity` : Fraction des éléments ou des lignes à supprimer (exprimée entre 0 et 1).

##### <a id='toc8_1_1_1_2_'></a>[**Sorties attendues** :](#toc0_)
- `pruned_matrix` : Matrice après élagage, de la même taille que la matrice d'origine, mais avec certains éléments ou lignes mises à zéro.
- `mask` : Matrice ou vecteur binaire indiquant les positions des éléments ou lignes conservées.

</div>

</div>

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Generate a random 2D matrix
rows, cols = 10,10
random_matrix = np.random.uniform(low=-1, high=1, size=(rows, cols))
target_sparsity = 0.5  # Target 50% sparsity


def prune_L1_elements(matrix, target_sparsity):
    """
    Implémentez l'élagage en fonction de la norme L1 élémentaire.
    """

def prune_L1_rows(matrix, target_sparsity):
    """
    Implémentez l'élagage en fonction de la norme L1 par ligne.
    """

def prune_L2_rows(matrix, target_sparsity):
    """
    Implémentez l'élagage en fonction de la norme L2 par ligne.
    """


In [ ]:
def prune_L1_elements(matrix, target_sparsity):
    """
    Prune elements of the matrix based on L1 norm (absolute values).
    Args:
        matrix: NumPy 2D array to prune.
        target_sparsity: Fraction of elements to set to zero.
    Returns:
        pruned_matrix: Matrix with pruned elements.
        mask: Binary mask indicating retained elements.
    """
    flat_matrix = np.abs(matrix.flatten())
    num_to_prune = int(flat_matrix.size * target_sparsity)
    threshold = np.partition(flat_matrix, num_to_prune)[num_to_prune]
    mask = np.abs(matrix) >= threshold
    pruned_matrix = matrix * mask
    return pruned_matrix, mask

def prune_L1_rows(matrix, target_sparsity):
    """
    Prune rows of the matrix based on the L1 norm (sum of absolute values per row).
    Args:
        matrix: NumPy 2D array to prune.
        target_sparsity: Fraction of rows to set to zero.
    Returns:
        pruned_matrix: Matrix with pruned rows.
        mask: Binary mask indicating retained rows.
    """
    row_norms = np.sum(np.abs(matrix), axis=1)
    num_to_prune = int(row_norms.size * target_sparsity)
    threshold = np.partition(row_norms, num_to_prune)[num_to_prune]
    mask = row_norms >= threshold
    pruned_matrix = matrix * mask[:, np.newaxis]
    return pruned_matrix, mask

def prune_L2_rows(matrix, target_sparsity):
    """
    Prune rows of the matrix based on the L2 norm (Euclidean norm per row).
    Args:
        matrix: NumPy 2D array to prune.
        target_sparsity: Fraction of rows to set to zero.
    Returns:
        pruned_matrix: Matrix with pruned rows.
        mask: Binary mask indicating retained rows.
    """
    row_norms = np.linalg.norm(matrix, axis=1, ord=2)
    num_to_prune = int(row_norms.size * target_sparsity)
    threshold = np.partition(row_norms, num_to_prune)[num_to_prune]
    mask = row_norms >= threshold
    pruned_matrix = matrix * mask[:, np.newaxis]
    return pruned_matrix, mask

<div style="background-color:#eaf6fb; border-radius:8px; padding:20px; border-left:6px solid #4682b4; color:black; font-family:Arial, sans-serif;">


<div style="background-color: #e6f7ff; border-radius:8px; padding:15px; border-left:6px solid #1e90ff; color:black; font-family:Arial, sans-serif;">

### <a id='toc8_1_2_'></a>[<img src="https://img.icons8.com/ios-filled/20/1e90ff/info--v1.png" style="vertical-align:middle;"/> **INFO :**](#toc0_)

1. **`plot_matrices`** : Affiche côte à côte les matrices originale et élaguée, en mettant en évidence les éléments mis à zéro pour comparaison.

2. **`compute_sparsity`** : Calcule la parcimonie d'une matrice comme la proportion d'éléments nuls par rapport au nombre total d'éléments.

3. **`plot_weight_histograms`** : Trace des histogrammes côte à côte des distributions de poids dans les matrices originale et élaguée pour visualiser l'effet de l'élagage.

</div>
</div>

In [ ]:
# Plot the original and pruned matrices
def plot_matrices(original_matrix, pruned_matrix):
    """
    Plot the original and pruned matrices side-by-side.
    Args:
        original_matrix: The original 2D NumPy array.
        pruned_matrix: The pruned 2D NumPy array.
    """
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    
    def plot_matrix(matrix, title, ax):
        """Helper function to plot a single matrix."""
        ax.imshow(matrix == 0, vmin=0, vmax=1, cmap='tab20c')
        ax.set_title(title)
        ax.axis('off')
        for i in range(matrix.shape[0]):
            for j in range(matrix.shape[1]):
                ax.text(j, i, f'{matrix[i, j]:.2f}', ha="center", va="center", color="k")
    
    # Plot the matrices
    plot_matrix(original_matrix, "Original Matrix", axes[0])
    plot_matrix(pruned_matrix, "Pruned Matrix", axes[1])
    
    plt.tight_layout()
    plt.show()

# Compute sparsity
def compute_sparsity(matrix):
    """
    Calculate the sparsity of a matrix.
    Sparsity = (# zeros / # total elements)
    """
    return 1.0 - np.count_nonzero(matrix) / matrix.size


# Function to plot histograms of weights
def plot_weight_histograms(original_matrix, pruned_matrix, title_original="Original Weights", title_pruned="Pruned Weights"):
    """
    Plot the histograms of weights in the original and pruned matrices.
    Args:
        original_matrix: The original 2D matrix before pruning.
        pruned_matrix: The pruned 2D matrix.
        title_original: Title for the histogram of the original weights.
        title_pruned: Title for the histogram of the pruned weights.
    """
    # Flatten matrices to 1D arrays
    original_weights = original_matrix.flatten()
    pruned_weights = pruned_matrix.flatten()

    # Plot histograms side by side
    fig, axes = plt.subplots(1, 2, figsize=(12, 5), sharey=True)

    # Original weights histogram
    axes[0].hist(original_weights, bins=256, color='blue', alpha=0.7)
    axes[0].set_title(title_original)
    axes[0].set_xlabel("Weight Value")
    axes[0].set_ylabel("Frequency")

    # Pruned weights histogram
    axes[1].hist(pruned_weights, bins=50, color='green', alpha=0.7)
    axes[1].set_title(title_pruned)
    axes[1].set_xlabel("Weight Value")

    plt.tight_layout()
    plt.show()

<div style="background-color:#eaf6fb; border-radius:8px; padding:20px; border-left:6px solid #4682b4; color:black; font-family:Arial, sans-serif;">

<div style="background-color:#fae1e1; border-radius:8px; padding:15px; border-left:6px solid #bf2929; color:black; font-family:Arial, sans-serif;">

### <a id='toc8_1_3_'></a>[**Consigne : Appliquer l'élagage fine-grainé**](#toc0_)

1. Appliquez les trois méthodes d'élagage : $L_1$ élémentaire, $L_1$ par ligne, et une norme $L_p$ générale (par exemple $L_2$).  
2. Visualisez les matrices originale et élaguée côte à côte pour chaque méthode d'élagage.  
3. Comparez les distributions des poids avant et après élagage en traçant des histogrammes.  
4. Calculez et affichez les statistiques de parcimonie pour la matrice originale, la matrice élaguée et le masque.  
5. Analysez l'impact de chaque méthode sur la parcimonie et la structure de la matrice.  

</div></div>

In [ ]:
# Apply fine-grained pruning
# L1 norm pruning
pruned_matrix_L1, mask_L1 = prune_L1_elements(random_matrix, target_sparsity)

# L2 norm pruning
pruned_matrix_L2, mask_L2 = prune_L1_rows(random_matrix, target_sparsity)

# General L_p norm pruning (e.g., p=3)
pruned_matrix_Lp, mask_Lp = prune_L2_rows(random_matrix, target_sparsity)

plot_matrices(random_matrix, pruned_matrix_L1)
plot_matrices(random_matrix, pruned_matrix_L2)
plot_matrices(random_matrix, pruned_matrix_Lp)

plot_weight_histograms(random_matrix, pruned_matrix_L1)

# Print statistics
print(f"Original sparsity: {compute_sparsity(random_matrix):.2f}")
print(f"Pruned sparsity: {compute_sparsity(pruned_matrix_L1):.2f}")
print(f"Mask sparsity: {compute_sparsity(mask_L1):.2f}")

<div style="background-color:#eaf6fb; border-radius:8px; padding:20px; border-left:6px solid #4682b4; color:black; font-family:Arial, sans-serif;">


<div style="background-color:#fae1e1; border-radius:8px; padding:15px; border-left:6px solid #bf2929; color:black; font-family:Arial, sans-serif;">

### <a id='toc8_1_4_'></a>[<img src="https://img.icons8.com/ios-filled/20/bf2929/error--v1.png" style="vertical-align:middle;"/> **Consigne : Pourcentage Moyen de Zéros (PMZ)**](#toc0_)




1. Implémentez une fonction **`compute_apoz`** qui calcule le **Pourcentage Moyen de Zéros (PMZ)** pour chaque canal d'un tenseur d'activations.  
2. Créez une fonction **`prune_channels_based_on_apoz`** pour élaguer les canaux ayant un **APoZ** supérieur à un seuil donné, en mettant à zéro les canaux correspondants.  
3. Affichez les valeurs de **APoZ** par canal et identifiez les canaux élagués.  
4. Testez les fonctions avec un exemple de tenseur d'activations et vérifiez que les canaux non pertinents sont correctement élagués.  

</div></div>

In [ ]:
def compute_apoz(activations):
    """
    Compute the Average Percentage of Zeros (APoZ) for each channel in the activation tensor.
    """
    batch_size, num_channels, height, width = activations.shape
    total_elements_per_channel = batch_size * height * width
    zeros_per_channel = np.sum(activations == 0, axis=(0, 2, 3))
    apoz = zeros_per_channel / total_elements_per_channel
    return apoz

def prune_channels_based_on_apoz(apoz, activations, threshold=0.5):
    """
    Prune channels based on the APoZ threshold.
    """
    print("APoZ values per channel:", apoz)  # Debug: Print APoZ values
    pruned_channels = apoz > threshold
    print("Pruned Channels:", pruned_channels)  # Debug: Check which channels are pruned
    pruned_activations = np.copy(activations)
    pruned_activations[:, pruned_channels, :, :] = 0  # Set pruned channels to zero
    return pruned_activations, pruned_channels


<div style="background-color:#eaf6fb; border-radius:8px; padding:20px; border-left:6px solid #4682b4; color:black; font-family:Arial, sans-serif;">

<div style="background-color: #e6f7ff; border-radius:8px; padding:15px; border-left:6px solid #1e90ff; color:black; font-family:Arial, sans-serif;">

### <a id='toc8_1_5_'></a>[<img src="https://img.icons8.com/ios-filled/20/1e90ff/info--v1.png" style="vertical-align:middle;"/> **INFO :**](#toc0_)

La fonction **`visualize_all_batches_and_channels`** permet de visualiser les activations originales et élaguées de tous les lots (batches) et canaux.  
Elle organise les activations originales sur la première rangée et les activations élaguées sur la deuxième, tout en ajoutant des titres pour chaque canal et lot, et une légende décrivant leur état (original ou élagué).

</div> </div>

In [6]:
def visualize_all_batches_and_channels(activations, pruned_activations, apoz, pruned_channels):
    """
    Visualize all original activations for all batches and channels first,
    followed by all pruned activations in the same layout.
    """
    num_batches, num_channels, height, width = activations.shape

    # Total rows: 2 (Original + Pruned), Columns: num_batches * num_channels
    fig, axes = plt.subplots(
        2, num_batches * num_channels, figsize=(3 * num_batches * num_channels, 6)
    )

    # Plot original activations
    for batch_idx in range(num_batches):
        for channel_idx in range(num_channels):
            ax_original = axes[0, batch_idx * num_channels + channel_idx]
            ax_original.imshow(
                activations[batch_idx, channel_idx],
                cmap="viridis",
                interpolation="nearest"
            )
            ax_original.set_title(f"Batch {batch_idx} - Channel {channel_idx}")
            ax_original.axis("off")

    # Plot pruned activations
    for batch_idx in range(num_batches):
        for channel_idx in range(num_channels):
            ax_pruned = axes[1, batch_idx * num_channels + channel_idx]
            ax_pruned.imshow(
                pruned_activations[batch_idx, channel_idx],
                cmap="viridis",
                interpolation="nearest"
            )
            ax_pruned.set_title(f"Pruned Batch {batch_idx} - Channel {channel_idx}")
            ax_pruned.axis("off")

    # Adjust layout
    axes[0, 0].set_ylabel("Original Activations", fontsize=14)
    axes[1, 0].set_ylabel("Pruned Activations", fontsize=14)
    plt.tight_layout()
    plt.show()

    # Display APoZ values with pruning results
    print("\nAPoZ per Channel and Pruning Results:")
    for i, value in enumerate(apoz):
        status = "Pruned" if pruned_channels[i] else "Retained"
        print(f"Channel {i}: APoZ = {value:.2f} -> {status}")

In [7]:
# EXEMPLE De COURS
activations = np.array([
    [
        [[0, 0.1, 0.5, 1], [1.2, 0.6, 0.3, 0.2], [0, 0.5, 0.3, 0], [0.2, 0, 0.8, 0]],
        [[0.1, 0.5, 0, 0], [0.2, 0.3, 0, 1], [1.2, 1, 0.2, 0.3], [0.1, 0.6, 0.7, 0.1]],
        [[0.5, 0, 0.2, 0.1], [0, 0.2, 0, 0], [0, 0.8, 0.1, 0], [0.2, 0, 0.3, 0]]
    ],
    [
        [[0.2, 0.5, 0, 0], [0, 0.2, 0.1, 0], [0, 0.8, 0, 0.1], [0.1, 0.4, 0.5, 0]],
        [[0, 0.2, 0, 0.1], [0, 0.8, 0, 1], [0.2, 0, 0, 0.3], [0.2, 0.3, 0.5, 0]],
        [[0.5, 0, 0.1, 0.1], [0.2, 0, 0, 0], [0.8, 0, 0, 0.1], [0.2, 0, 0.3, 0]]
    ]
])

In [ ]:
# Simulate activations with shape (batch=2, channels=3, height=4, width=4)
activations = np.random.rand(2, 3, 4, 4)  # Example 4D tensor (batch=2, channels=3, height=4, width=4)
activations[activations < 0.2] = 0  # Introduce some zeros

<div style="background-color:#eaf6fb; border-radius:8px; padding:20px; border-left:6px solid #4682b4; color:black; font-family:Arial, sans-serif;">


<div style="background-color:#fae1e1; border-radius:8px; padding:15px; border-left:6px solid #bf2929; color:black; font-family:Arial, sans-serif;">

### <a id='toc8_1_6_'></a>[<img src="https://img.icons8.com/ios-filled/20/bf2929/error--v1.png" style="vertical-align:middle;"/> **Consigne :**](#toc0_)

Appliquez *Pourcentage Moyen de Zéros (APoZ)** pour chaque canal et élaguez les canaux ayant un **APoZ** supérieur à un seuil donné.  
Ajoutez une visualisation (`visualize_all_batches_and_channels`) pour comparer les activations originales et élaguées pour tous les lots et canaux.

</div></div>

In [ ]:
# Compute APoZ
apoz = compute_apoz(activations)

# Prune channels with APoZ > threshold
threshold = 0.2  # Lower threshold to ensure some pruning
pruned_activations, pruned_channels = prune_channels_based_on_apoz(apoz, activations, threshold)

# Visualize results
visualize_all_batches_and_channels(activations, pruned_activations, apoz, pruned_channels)

  ---
### <a id='toc1_1_1_'></a>[**Table des matières**](#toc0_)


- [ 🧠  TP2 - Elagage (Pruning) des modèles IA ](#toc1_)    
    
- [1. Introduction à l'élagage (Pruning)](#toc2_)    
  - [1.1 Techniques d'élagage ](#toc3_)    
        - [**Impact sur les performances**](#toc3_1_1_1_)    
  - [1.2 Granularités d’élagage ](#toc4_)    
  - [1.3 Critères d'élagage](#toc5_)    
    - [a) Élagage basé sur l'amplitude ](#toc6_)    
    - [b) Élagage Basé sur le Pourcentage Moyen de Zéros (PMZ) - Average Percentage of Zeros (APoZ)](#toc7_)    
- [🔍 2. Échauffement : Bases de l'Elagage](#toc8_)    

  
- [🖼️ 3. Exploration de l'élagage des modèles IA](#toc9_)    
  - [3.1 Chargement des données](#toc10_)    
  - [3.2 Construire un modèle](#toc11_)    
  - [3.3 Entraîner le modèle sans élagage](#toc12_)      
  - [3.4. Évaluation de la Précision du modèle de base  📊](#toc13_)    
  - [3.4 Appliquer l'Élagage](#toc14_)    
  - [3.5 Compresser le modèle élagué](#toc15_)    
  - [3.6  Combinez l'élagage (**pruning**) et la quantification (**quantization**) pour créer un modèle 10 fois plus petit.](#toc17_)    

- [✅  4. Conslusion](#toc19_)    

- [🎁 5. BONUS](#toc20_)    

  ---

<!-- vscode-jupyter-toc-config
    numbering=false
    anchor=true
    flat=false
    minLevel=1
    maxLevel=6
    /vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

<div style="background-color:#eaf6fb; border-radius:8px; padding:20px; border-left:6px solid #4682b4; color:black; font-family:Arial, sans-serif;">

# <a id='toc9_'></a>[<div style="text-align:center; border-radius:8px; padding:8px; color:white; margin:10px 0; font-size:100%; font-family:Arial, sans-serif; background-color:#4682b4;"><b>🖼️  3. Exploration de l'élagage des modèles IA</b></div>](#toc0_)





<div style="background-color: #e6f7ff; border-radius:8px; padding:15px; border-left:6px solid #1e90ff; color:black; font-family:Arial, sans-serif;">

<img src="https://img.icons8.com/ios-filled/20/1e90ff/info--v1.png" style="vertical-align:middle;"/> **INFO :** Importation des bibliothèques et fixation de seeds



</div> </div>

In [18]:
# import libs and fix seed for reproducibility

import math
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.axes as axes
import numpy as np
import os
import pandas as pd
import seaborn as sns
import tempfile
import tensorboard
import tensorflow as tf
import timeit
import zipfile
import tensorflow as tf
import numpy as np
import tensorflow_model_optimization as tfmot
import tf_keras as keras


from IPython.core.pylabtools import figsize
from numpy import linalg as LA


from tensorflow_model_optimization.python.core.keras.compat import keras

plt.style.use('fivethirtyeight')
sns.set_context('notebook')
pd.set_option('display.max_rows', 30)
np.random.seed(1337)


<div style="background-color:#eaf6fb; border-radius:8px; padding:20px; border-left:6px solid #4682b4; color:black; font-family:Arial, sans-serif;">

<div style="background-color: #e6f7ff; border-radius:8px; padding:15px; border-left:6px solid #1e90ff; color:black; font-family:Arial, sans-serif;">

<img src="https://img.icons8.com/ios-filled/20/1e90ff/info--v1.png" style="vertical-align:middle;"/> **INFO :** Crée deux répertoires nommés images et models dans le système de fichiers courant pour stocker, respectivement, des images et des modèles.

</div> </div>

In [ ]:
!mkdir images
!mkdir models

<div style="background-color:#eaf6fb; border-radius:8px; padding:20px; border-left:6px solid #4682b4; color:black; font-family:Arial, sans-serif;">


<div style="background-color:#eaf6fb; border-radius:8px; padding:20px; border-left:6px solid #f17c12; color:black; font-family:Arial, sans-serif;">



# <a id='toc10_'></a>[<div style="text-align:left; border-radius:6px; padding:px; color:white; margin:5px 0; font-size:70%; font-family:Arial, sans-serif; background-color: #f17c12;"><b>3.1 Chargement des données](#toc0_)
 </b> 


<div style="background-color: #e6f7ff; border-radius:8px; padding:15px; border-left:6px solid #1e90ff; color:black; font-family:Arial, sans-serif;">


<img src="https://img.icons8.com/ios-filled/20/1e90ff/info--v1.png" style="vertical-align:middle;"/> **INFO : `load_dataset`**
La fonction  charge les données MNIST, les divise en ensembles d'entraînement et de test, puis les normalise entre 0 et 1.  
Elle retourne les caractéristiques (`x_train`, `x_test`) et les étiquettes (`y_train`, `y_test`) prêtes à être utilisées pour l'entraînement d'un modèle.
</div>
</div>
</div>

In [20]:
def load_dataset():
    """
    Loads and preprocesses the data for this task.
    Args:
      None
    Returns:
      x_train: Features for training data 
      x_test: Features for test data
      y_train: Labels for training data
      y_test: Labels for test data
    """    
    # the data, shuffled and split between train and test sets
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    # Convert to float32 and normalize
    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0
    return x_train, x_test, y_train, y_test



In [ ]:
# We will load separate tensors for the MNIST and FMNIST data. 
mnist_x_train, mnist_x_test, mnist_y_train, mnist_y_test = load_dataset()

<div style="background-color:#eaf6fb; border-radius:8px; padding:20px; border-left:6px solid #4682b4; color:black; font-family:Arial, sans-serif;">


<div style="background-color:#eaf6fb; border-radius:8px; padding:20px; border-left:6px solid #f17c12; color:black; font-family:Arial, sans-serif;">

# <a id='toc11_'></a>[<div style="text-align:left; border-radius:6px; padding:px; color:white; margin:5px 0; font-size:70%; font-family:Arial, sans-serif; background-color: #f17c12;"><b>3.2 Construire un modèle](#toc0_)
 </b> 


<div style="background-color:#fae1e1; border-radius:8px; padding:15px; border-left:6px solid #bf2929; color:black; font-family:Arial, sans-serif;">

### <a id='toc11_1_1_'></a>[<img src="https://img.icons8.com/ios-filled/20/bf2929/error--v1.png" style="vertical-align:middle;"/> **Consigne :**](#toc0_)


1. Implémentez une fonction **`build_model_arch`** qui définit une architecture de modèle simple avec des couches convolutives, de pooling et entièrement connectées.  
2. La fonction doit inclure un reshape des données d'entrée pour correspondre à la forme `(28, 28, 1)` des images MNIST.  
3. Ajoutez une couche de convolution suivie d'une couche de pooling, puis une couche `Flatten` et une couche finale `Dense` pour la classification en 10 classes.  
4. Retournez le modèle non compilé, prêt à être utilisé pour l'entraînement.

</div>
</div>
</div>





In [22]:
def build_model_arch():
  """
  Builds the model architecture
  """
  # Define the model architecture.
  model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10)])
  return model

In [23]:
mnist_model_base = build_model_arch()

<div style="background-color:#eaf6fb; border-radius:8px; padding:20px; border-left:6px solid #4682b4; color:black; font-family:Arial, sans-serif;">


<div style="background-color:#eaf6fb; border-radius:8px; padding:20px; border-left:6px solid #f17c12; color:black; font-family:Arial, sans-serif;">


<div style="background-color: #e6f7ff; border-radius:8px; padding:15px; border-left:6px solid #1e90ff; color:black; font-family:Arial, sans-serif;">


<img src="https://img.icons8.com/ios-filled/20/1e90ff/info--v1.png" style="vertical-align:middle;"/> **INFO :** Ce code crée un répertoire temporaire pour stocker les journaux d'entraînement du modèle.   La fonction `tempfile.mkdtemp()` génère un dossier temporaire unique, et son chemin est affiché à l'aide de `print` :
</div>
</div>
</div>
</div>

In [ ]:

# Define the path for the 'temp' directory in the current workspace
logdir_base = os.path.join("", "temp_baseline")

# Create the directory if it doesn't already exist
os.makedirs(logdir_base, exist_ok=True)

print('Writing training logs to ' + logdir_base)

<div style="background-color:#eaf6fb; border-radius:8px; padding:20px; border-left:6px solid #4682b4; color:black; font-family:Arial, sans-serif;">


<div style="background-color:#eaf6fb; border-radius:8px; padding:20px; border-left:6px solid #f17c12; color:black; font-family:Arial, sans-serif;">

# <a id='toc12_'></a>[<div style="text-align:left; border-radius:6px; padding:px; color:white; margin:5px 0; font-size:70%; font-family:Arial, sans-serif; background-color: #f17c12;"><b>3.3 Entraîner le modèle sans élagage](#toc0_)
 </b> 

<div style="background-color:#fae1e1; border-radius:8px; padding:15px; border-left:6px solid #bf2929; color:black; font-family:Arial, sans-serif;">

### <a id='toc12_1_1_'></a>[<img src="https://img.icons8.com/ios-filled/20/bf2929/error--v1.png" style="vertical-align:middle;"/> **Consigne :**](#toc0_)

1. Implémentez une fonction **`compile_fit_model`** pour compiler et entraîner un modèle de classification en utilisant l'optimiseur **Adam** et une perte de type `SparseCategoricalCrossentropy`.  
2. La fonction doit inclure une validation sur 10% des données d'entraînement via le paramètre `validation_split`.  
3. Appliquez cette fonction pour entraîner le modèle MNIST, en spécifiant un nombre d'époques et en affichant le résumé de chaque modèle après l'entraînement.  




</div>
</div>
</div>

</div>




In [25]:
def compile_fit_model(model, x_train, y_train, epochs):
    """
    Training our original model, pre-pruning
    Args:
      model: Uncompiled Keras model
      x_train: Features for training data 
      y_train: Labels for training data
      epochs: Number of epochs for training

    Returns:
      model: compiled model :  return a new model different from the input model (parm model will be used for comparison)
    """
 #   .... Tap your code here .....

In [ ]:
def compile_fit_model(model, x_train, y_train, epochs):
  # Train the digit classification model
  model.compile(optimizer='adam',
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  model.fit(
    x_train,
    y_train,
    epochs=epochs,
    validation_split=0.1,
  )
  return model

batch_size = 128
epochs = 2

mnist_model = compile_fit_model(mnist_model_base,
                                               mnist_x_train,
                                               mnist_y_train,
                                               epochs,
                                           )
print(mnist_model.summary())


<div style="background-color:#eaf6fb; border-radius:8px; padding:20px; border-left:6px solid #4682b4; color:black; font-family:Arial, sans-serif;">


<div style="background-color:#eaf6fb; border-radius:8px; padding:20px; border-left:6px solid #f17c12; color:black; font-family:Arial, sans-serif;">

# <a id='toc13_'></a>[<div style="text-align:left; border-radius:6px; padding:px; color:white; margin:5px 0; font-size:70%; font-family:Arial, sans-serif; background-color: #f17c12;"><b>3.4. Évaluation de la Précision du modèle de base  📊](#toc0_)
 </b> 

<div style="background-color:#fae1e1; border-radius:8px; padding:15px; border-left:6px solid #bf2929; color:black; font-family:Arial, sans-serif;">

### <a id='toc13_1_1_'></a>[<img src="https://img.icons8.com/ios-filled/20/bf2929/error--v1.png" style="vertical-align:middle;"/> **Consigne :**](#toc0_)

1. Évaluez la précision du modèle **MNIST** sur l'ensemble de test en utilisant la méthode `evaluate`.  
2. Affichez cette précision.



</div>
</div>
</div>

</div>




In [ ]:
_, baseline_model_accuracy = mnist_model.evaluate(
    mnist_x_test, mnist_y_test, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)


<div style="background-color:#eaf6fb; border-radius:8px; padding:20px; border-left:6px solid #4682b4; color:black; font-family:Arial, sans-serif;">


<div style="background-color:#eaf6fb; border-radius:8px; padding:20px; border-left:6px solid #f17c12; color:black; font-family:Arial, sans-serif;">


<div style="background-color: #e6f7ff; border-radius:8px; padding:15px; border-left:6px solid #1e90ff; color:black; font-family:Arial, sans-serif;">


<img src="https://img.icons8.com/ios-filled/20/1e90ff/info--v1.png" style="vertical-align:middle;"/> **INFO :** Ce code sauvegarde le modèle MNIST dans un fichier temporaire avec l'extension `.h5`, sans inclure l'optimiseur.  
La fonction `tempfile.mkstemp` génère un chemin unique, et le modèle est sauvegardé à cet emplacement à l'aide de `keras.models.save_model`. Le chemin du fichier sauvegardé est affiché pour vérification.

</div>
</div>
</div>
</div>

In [ ]:

_, keras_file = tempfile.mkstemp('.h5')
keras.models.save_model(mnist_model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

<div style="background-color:#eaf6fb; border-radius:8px; padding:20px; border-left:6px solid #4682b4; color:black; font-family:Arial, sans-serif;">


<div style="background-color:rgba(136, 125, 125, 0.15); border-radius:8px; padding:20px; border-left:6px solid #f17c12; color:black; font-family:Arial, sans-serif;">

# <a id='toc14_'></a>[<div style="text-align:left; border-radius:6px; padding:px; color:white; margin:5px 0; font-size:70%; font-family:Arial, sans-serif; background-color: #f17c12;"><b>3.4 Appliquer l'Élagage </b>](#toc0_)


Maintenant que le modèle est entraîné, nous allons appliquer une stratégie d'élagage pour mettre à zéro certains poids du modèle.

 </b> 

<div style="background-color:#fae1e1; border-radius:8px; padding:15px; border-left:6px solid #bf2929; color:black; font-family:Arial, sans-serif;">

### <a id='toc14_1_1_'></a>[<img src="https://img.icons8.com/ios-filled/20/bf2929/error--v1.png" style="vertical-align:middle;"/> **Consigne :**](#toc0_)

1. Utilisez la fonction **`prune_low_magnitude`** de TensorFlow Model Optimization pour appliquer l'élagage aux poids du modèle MNIST.  
2. Définissez un planning d'élagage progressif (`PolynomialDecay`) avec des paramètres tels qu'une parcimonie initiale de 50% et une parcimonie finale de 80% sur la durée de 10 époques.  
3. Recompilez le modèle élagué après l'application de `prune_low_magnitude`, en conservant l'optimiseur et les métriques utilisés dans le modèle original.  
4. Affichez un résumé du modèle élagué pour vérifier que les couches ont bien été modifiées pour inclure les wrappers d'élagage.



</div>
<div style="background-color: #e6f7ff; border-radius:8px; padding:15px; border-left:6px solid #1e90ff; color:black; font-family:Arial, sans-serif;">

### <a id='toc14_1_2_'></a>[<img src="https://img.icons8.com/ios-filled/20/1e90ff/info--v1.png" style="vertical-align:middle;"/> Ressource :](#toc0_)
Pour plus d'informations sur la fonction **`prune_low_magnitude`** et ses paramètres, consultez la documentation officielle de TensorFlow Model Optimization :   [Documentation officielle - prune_low_magnitude](https://www.tensorflow.org/model_optimization/api_docs/python/tfmot/sparsity/keras/prune_low_magnitude)

</div>

</div>

</div>




In [ ]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 10
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = mnist_x_train.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(mnist_model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()

<div style="background-color:#eaf6fb; border-radius:8px; padding:20px; border-left:6px solid #4682b4; color:black; font-family:Arial, sans-serif;">


<div style="background-color:#eaf6fb; border-radius:8px; padding:20px; border-left:6px solid #f17c12; color:black; font-family:Arial, sans-serif;">



<div style="background-color:#fae1e1; border-radius:8px; padding:15px; border-left:6px solid #bf2929; color:black; font-family:Arial, sans-serif;">

### <a id='toc14_1_3_'></a>[<img src="https://img.icons8.com/ios-filled/20/bf2929/error--v1.png" style="vertical-align:middle;"/> **Consigne :**](#toc0_)

1. Entraînez et évaluez le modèle élagué en le comparant avec les performances du modèle de base.  
2. Affinez le modèle élagué en utilisant les callbacks suivants :  
   - **`tfmot.sparsity.keras.UpdatePruningStep`** : pour mettre à jour les étapes d'élagage pendant l'entraînement.  
   - **`tfmot.sparsity.keras.PruningSummaries`** : pour générer des journaux permettant de suivre la progression de l'élagage et de déboguer.  
3. Configurez un répertoire temporaire pour stocker les journaux et suivez leur emplacement avec la fonction `print`.  
4. Effectuez l'entraînement avec le jeu de données MNIST, en spécifiant une division pour la validation (`validation_split`) et un ensemble de callbacks pour suivre les étapes d'élagage.




</div>
</div>
</div>

</div>





In [ ]:
logdir = tempfile.mkdtemp()
print(logdir)
callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

#Tap your code here : 

model_for_pruning.fit(mnist_x_train, mnist_y_train,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

<div style="background-color:#eaf6fb; border-radius:8px; padding:20px; border-left:6px solid #4682b4; color:black; font-family:Arial, sans-serif;">


<div style="background-color:#eaf6fb; border-radius:8px; padding:20px; border-left:6px solid #f17c12; color:black; font-family:Arial, sans-serif;">



<div style="background-color:#fae1e1; border-radius:8px; padding:15px; border-left:6px solid #bf2929; color:black; font-family:Arial, sans-serif;">

### <a id='toc14_1_4_'></a>[<img src="https://img.icons8.com/ios-filled/20/bf2929/error--v1.png" style="vertical-align:middle;"/> **Consigne :**](#toc0_)


1. Évaluez le modèle élagué **`model_for_pruning`** sur l'ensemble de test MNIST à l'aide de la méthode `evaluate`.  
2. Comparez la précision du modèle élagué avec celle du modèle de base en affichant les deux valeurs (`baseline_model_accuracy` et `model_for_pruning_accuracy`).  
3. Vérifiez que l'élagage n'a pas entraîné de perte significative de précision sur l'ensemble de test.

</div>
</div>
</div>

</div>





In [ ]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   mnist_x_test, mnist_y_test, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy) 
print('Pruned test accuracy:', model_for_pruning_accuracy)

In [ ]:
%tensorboard --logdir={logdir} --port=6001


<div style="background-color:#eaf6fb; border-radius:8px; padding:20px; border-left:6px solid #4682b4; color:black; font-family:Arial, sans-serif;">


<div style="background-color:rgba(136, 125, 125, 0.15); border-radius:8px; padding:20px; border-left:6px solid #f17c12; color:black; font-family:Arial, sans-serif;">

# <a id='toc15_'></a>[<div style="text-align:left; border-radius:6px; padding:px; color:white; margin:5px 0; font-size:70%; font-family:Arial, sans-serif; background-color: #f17c12;"><b>3.5 Compresser le modèle élagué </b>](#toc0_)




<div style="background-color:rgba(149, 177, 186, 0.2); border-radius:8px; padding:15px; border-left:6px solid #115175; color:black; font-family:Arial, sans-serif;">

# <a id='toc16_'></a>[<div style="text-align:left; border-radius:6px; padding:6px; color:white; margin:5px 0; font-size:50%; font-family:Arial, sans-serif; background-color: #115175;"><b> a)  3 X Plus pétit </b></div>](#toc0_)

 





<div style="background-color:#fae1e1; border-radius:8px; padding:15px; border-left:6px solid #bf2929; color:black; font-family:Arial, sans-serif;">

### <a id='toc16_1_1_'></a>[<img src="https://img.icons8.com/ios-filled/20/bf2929/error--v1.png" style="vertical-align:middle;"/> **Consigne :  📊**](#toc0_)

1. Créez des modèles **3 fois plus petits** en appliquant l'élagage et la compression.  
2. Utilisez **`tfmot.sparsity.keras.strip_pruning`** pour retirer les variables supplémentaires utilisées uniquement pendant l'entraînement, afin d'optimiser le modèle pour l'inférence.  
3. Appliquez un algorithme de compression standard (ex. **gzip**) pour compresser davantage les matrices de poids sérialisées, qui contiennent désormais de nombreux zéros grâce à l'élagage.  
4. Vérifiez et comparez les tailles des modèles d'origine, élagués, et compressés pour observer les bénéfices de la compression.  

</div>
<div style="background-color: #e6f7ff; border-radius:8px; padding:15px; border-left:6px solid #1e90ff; color:black; font-family:Arial, sans-serif;">

### <a id='toc16_1_2_'></a>[<img src="https://img.icons8.com/ios-filled/20/1e90ff/info--v1.png" style="vertical-align:middle;"/> Ressource :](#toc0_)
Pour plus d'informations sur la fonction **`tfmot.sparsity.keras.strip_pruning`** et ses paramètres, consultez la documentation officielle de TensorFlow Model Optimization :   [Documentation officielle - strip_pruning](https://www.tensorflow.org/model_optimization/api_docs/python/tfmot/sparsity/keras/strip_pruning)

</div>

</div>

</div>

</div>


In [ ]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)



In [ ]:
_, pruned_keras_file = tempfile.mkstemp('.h5')
keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

<div style="background-color:#eaf6fb; border-radius:8px; padding:20px; border-left:6px solid #4682b4; color:black; font-family:Arial, sans-serif;">


<div style="background-color:rgba(136, 125, 125, 0.15); border-radius:8px; padding:20px; border-left:6px solid #f17c12; color:black; font-family:Arial, sans-serif;">


<div style="background-color:rgba(149, 177, 186, 0.2); border-radius:8px; padding:15px; border-left:6px solid #115175; color:black; font-family:Arial, sans-serif;">

<div style="background-color:#fae1e1; border-radius:8px; padding:15px; border-left:6px solid #bf2929; color:black; font-family:Arial, sans-serif;">

### <a id='toc16_1_3_'></a>[<img src="https://img.icons8.com/ios-filled/20/bf2929/error--v1.png" style="vertical-align:middle;"/> **Consigne :**](#toc0_)

1. Convertissez le modèle élagué exporté en un modèle **TFLite** en utilisant **`tf.lite.TFLiteConverter.from_keras_model`**.  
2. Générez un fichier TFLite à partir du modèle converti et sauvegardez-le dans un répertoire temporaire à l'aide de **`tempfile.mkstemp`**.  
3. Enregistrez le modèle TFLite en écrivant les données converties dans un fichier avec l'extension `.tflite`.  
4. Affichez le chemin du fichier TFLite sauvegardé pour validation.  

</div>

</div>

</div>

</div>




In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model = converter.convert()

_, pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)

<div style="background-color:#eaf6fb; border-radius:8px; padding:20px; border-left:6px solid #4682b4; color:black; font-family:Arial, sans-serif;">


<div style="background-color:#eaf6fb; border-radius:8px; padding:20px; border-left:6px solid #f17c12; color:black; font-family:Arial, sans-serif;">


<div style="background-color:rgba(149, 177, 186, 0.2); border-radius:8px; padding:15px; border-left:6px solid #115175; color:black; font-family:Arial, sans-serif;">
<div style="background-color: #e6f7ff; border-radius:8px; padding:15px; border-left:6px solid #1e90ff; color:black; font-family:Arial, sans-serif;">


<img src="https://img.icons8.com/ios-filled/20/1e90ff/info--v1.png" style="vertical-align:middle;"/> **INFO :** La fonction **`get_gzipped_model_size`** permet de compresser un fichier modèle à l'aide de l'algorithme **gzip** et de mesurer sa taille une fois compressé.   Elle crée un fichier temporaire `.zip` en utilisant **`zipfile.ZipFile`** avec la compression activée, et retourne la taille du fichier compressé en octets. 

</div>
</div>
</div>
</div>

In [48]:
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [ ]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))

<div style="background-color:#eaf6fb; border-radius:8px; padding:20px; border-left:6px solid #4682b4; color:black; font-family:Arial, sans-serif;">


<div style="background-color:rgba(136, 125, 125, 0.15); border-radius:8px; padding:20px; border-left:6px solid #f17c12; color:black; font-family:Arial, sans-serif;">

# <a id='toc17_'></a>[<div style="text-align:left; border-radius:6px; padding:px; color:white; margin:5px 0; font-size:70%; font-family:Arial, sans-serif; background-color: #f17c12;"><b>3.6  Combinez l'élagage (**pruning**) et la quantification (**quantization**) pour créer un modèle 10 fois plus petit.   </b>](#toc0_)




<div style="background-color:rgba(149, 177, 186, 0.2); border-radius:8px; padding:15px; border-left:6px solid #115175; color:black; font-family:Arial, sans-serif;">

# <a id='toc18_'></a>[<div style="text-align:left; border-radius:6px; padding:6px; color:white; margin:5px 0; font-size:50%; font-family:Arial, sans-serif; background-color: #115175;"><b> a)  10 X Plus pétit </b></div>](#toc0_)







<div style="background-color:#fae1e1; border-radius:8px; padding:15px; border-left:6px solid #bf2929; color:black; font-family:Arial, sans-serif;">

### <a id='toc18_1_1_'></a>[<img src="https://img.icons8.com/ios-filled/20/bf2929/error--v1.png" style="vertical-align:middle;"/> **Consigne : 📊**](#toc0_)

1. Appliquez une quantification post-entraînement au modèle élagué en utilisant l'API **TensorFlow Lite**.  
2. Convertissez le modèle élagué et quantifié en un modèle TFLite prêt pour une utilisation sur des appareils contraints.  
3. Comparez les tailles du modèle d'origine, du modèle élagué, et du modèle élagué + quantifié pour analyser les gains.

</div>
<div style="background-color: #e6f7ff; border-radius:8px; padding:15px; border-left:6px solid #1e90ff; color:black; font-family:Arial, sans-serif;">


### <a id='toc18_1_2_'></a>[<img src="https://img.icons8.com/ios-filled/20/1e90ff/info--v1.png" style="vertical-align:middle;"/> Ressource :](#toc0_)

Pour en savoir plus sur la quantification post-entraînement et ses bénéfices, consultez la documentation officielle de TensorFlow :  [Documentation officielle - Quantification post-entraînement](https://www.tensorflow.org/model_optimization/guide/quantization/post_training?hl=fr)

</div>

</div>

</div>






In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_and_pruned_tflite_model = converter.convert()

_, quantized_and_pruned_tflite_file = tempfile.mkstemp('.tflite')



with open(quantized_and_pruned_tflite_file, 'wb') as f:
  f.write(quantized_and_pruned_tflite_model)

print('Saved quantized and pruned TFLite model to:', quantized_and_pruned_tflite_file)

print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned and quantized TFlite model: %.2f bytes" % (get_gzipped_model_size(quantized_and_pruned_tflite_file)))

<div style="background-color:#eaf6fb; border-radius:8px; padding:20px; border-left:6px solid #4682b4; color:black; font-family:Arial, sans-serif;">


<div style="background-color:rgba(136, 125, 125, 0.15); border-radius:8px; padding:20px; border-left:6px solid #f17c12; color:black; font-family:Arial, sans-serif;">




<div style="background-color:rgba(149, 177, 186, 0.2); border-radius:8px; padding:15px; border-left:6px solid #115175; color:black; font-family:Arial, sans-serif;">



<div style="background-color:#fae1e1; border-radius:8px; padding:15px; border-left:6px solid #bf2929; color:black; font-family:Arial, sans-serif;">

### <a id='toc18_1_3_'></a>[<img src="https://img.icons8.com/ios-filled/20/bf2929/error--v1.png" style="vertical-align:middle;"/> **Consigne : Vérifiez la persistance de la précision entre TensorFlow et TFLite.  📊**](#toc0_)

1. Comparez les tailles du modèle d'origine, du modèle élagué, et du modèle élagué + quantifié pour analyser les gains.

</div>
<div style="background-color: #e6f7ff; border-radius:8px; padding:15px; border-left:6px solid #1e90ff; color:black; font-family:Arial, sans-serif;">

### <a id='toc18_1_4_'></a>[<img src="https://img.icons8.com/ios-filled/20/1e90ff/info--v1.png" style="vertical-align:middle;"/> INFO :](#toc0_)
La fonction **`evaluate_model`** permet d'évaluer un modèle TFLite sur un ensemble de test en utilisant un interpréteur TensorFlow Lite.  
Elle exécute une inférence sur chaque image de l'ensemble de test, en appliquant un prétraitement et un post-traitement adaptés au format d'entrée et de sortie du modèle.  
Les prédictions du modèle sont comparées aux labels réels pour calculer une précision moyenne.  
Cette fonction est utile pour vérifier que les performances du modèle optimisé (élagué et quantifié) restent proches de celles du modèle TensorFlow original.

</div>

</div>

</div>






In [55]:
import numpy as np

def evaluate_model(interpreter, test_images, test_labels):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on ever y image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels).mean()
  return accuracy

In [ ]:
interpreter = tf.lite.Interpreter(model_content=quantized_and_pruned_tflite_model)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter, mnist_x_test, mnist_y_test)

print('Pruned and quantized TFLite test_accuracy:', test_accuracy)
print('Pruned TF test accuracy:', model_for_pruning_accuracy)

  ---
### <a id='toc1_1_1_'></a>[**Table des matières**](#toc0_)


- [ 🧠  TP2 - Elagage (Pruning) des modèles IA ](#toc1_)    
    
- [1. Introduction à l'élagage (Pruning)](#toc2_)    
  - [1.1 Techniques d'élagage ](#toc3_)    
        - [**Impact sur les performances**](#toc3_1_1_1_)    
  - [1.2 Granularités d’élagage ](#toc4_)    
  - [1.3 Critères d'élagage](#toc5_)    
    - [a) Élagage basé sur l'amplitude ](#toc6_)    
    - [b) Élagage Basé sur le Pourcentage Moyen de Zéros (PMZ) - Average Percentage of Zeros (APoZ)](#toc7_)    
- [🔍 2. Échauffement : Bases de l'Elagage](#toc8_)    

  
- [🖼️ 3. Exploration de l'élagage des modèles IA](#toc9_)    
  - [3.1 Chargement des données](#toc10_)    
  - [3.2 Construire un modèle](#toc11_)    
  - [3.3 Entraîner le modèle sans élagage](#toc12_)      
  - [3.4. Évaluation de la Précision du modèle de base  📊](#toc13_)    
  - [3.4 Appliquer l'Élagage](#toc14_)    
  - [3.5 Compresser le modèle élagué](#toc15_)    
  - [3.6  Combinez l'élagage (**pruning**) et la quantification (**quantization**) pour créer un modèle 10 fois plus petit.](#toc17_)    

- [✅  4. Conslusion](#toc19_)    

- [🎁 5. BONUS](#toc20_)    

  ---

<!-- vscode-jupyter-toc-config
    numbering=false
    anchor=true
    flat=false
    minLevel=1
    maxLevel=6
    /vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

<div style="background-color:#eaf6fb; border-radius:8px; padding:20px; border-left:6px solid #4682b4; color:black; font-family:Arial, sans-serif;">

# <a id='toc19_'></a>[<div style="text-align:center; border-radius:8px; padding:8px; color:white; margin:10px 0; font-size:100%; font-family:Arial, sans-serif; background-color:#4682b4;"><b>✅ 4. Conslusion</b></div>](#toc0_)





Dans ce tutoriel, vous avez appris à créer des modèles et les élaguer en utilisant l'API TensorFlow Model Optimization Toolkit, à la fois pour TensorFlow et TFLite.   En combinant l'élagage avec la quantification post-entraînement, vous avez obtenu des bénéfices supplémentaires.  

Vous avez créé un modèle 10 fois plus petit pour MNIST, avec une différence minimale de précision.  


</div></div>

<div style="background-color:#eaf6fb; border-radius:8px; padding:20px; border-left:6px solid #4682b4; color:black; font-family:Arial, sans-serif;">

# <a id='toc20_'></a>[<div style="text-align:center; border-radius:8px; padding:8px; color:white; margin:10px 0; font-size:100%; font-family:Arial, sans-serif; background-color:#4682b4;"><b> 🎁 5. BONUS</b></div>](#toc0_)



<a href="https://colab.research.google.com/github/bouzid-s/IASSC/blob/main/TP2_Bonus_Model_pruning_exploration.ipynb#" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" style="width:300px;"/>
</a>
